In [1]:
import sys; sys.path.append('/fs/pool/pool-marsot')

In [2]:
%cd /fs/pool/pool-marsot/tankbind_philip/TankBind/tankbind/

/fs/gpfs41/lv11/fileset01/pool/pool-marsot/tankbind_philip/TankBind/tankbind


In [4]:
from tankbind_philip.TankBind.tankbind.evaluation_fast import evaluate_model_val

In [5]:
from tankbind_philip.TankBind.tankbind.model import TankBind, TankBindModel

In [6]:
model = TankBind.load_from_checkpoint("/fs/pool/pool-marsot/tankbind_philip/TankBind/results/TankBind/epoch=59-step=36000.ckpt")

In [7]:
model.to("cuda:0")

TankBind(
  (model): TankBindModel(
    (layernorm): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
    (conv_protein): GVP_embedding(
      (W_s): Embedding(20, 20)
      (W_v): Sequential(
        (0): LayerNorm(
          (scalar_norm): LayerNorm((26,), eps=1e-05, elementwise_affine=True)
        )
        (1): GVP(
          (wh): Linear(in_features=3, out_features=16, bias=False)
          (ws): Linear(in_features=42, out_features=128, bias=True)
          (wv): Linear(in_features=16, out_features=16, bias=False)
        )
      )
      (W_e): Sequential(
        (0): LayerNorm(
          (scalar_norm): LayerNorm((32,), eps=1e-05, elementwise_affine=True)
        )
        (1): GVP(
          (wh): Linear(in_features=1, out_features=1, bias=False)
          (ws): Linear(in_features=33, out_features=32, bias=True)
          (wv): Linear(in_features=1, out_features=1, bias=False)
        )
      )
      (layers): ModuleList(
        (0-2): 3 x GVPConvLayer(
          (conv): 

In [8]:
df = evaluate_model_val(model,)

['/fs/pool/pool-marsot/tankbind_philip/TankBind/dataset/val_dataset/processed/data.pt', '/fs/pool/pool-marsot/tankbind_philip/TankBind/dataset/val_dataset/processed/protein.pt', '/fs/pool/pool-marsot/tankbind_philip/TankBind/dataset/val_dataset/processed/compound.pt']
predicting pairwise distances


 23%|███████████████████████████▋                                                                                             | 219/959 [00:00<00:02, 303.52it/s][16:27:45] Unusual charge on atom 1 number of radical electrons set to zero
[16:27:45] Unusual charge on atom 1 number of radical electrons set to zero
 70%|█████████████████████████████████████████████████████████████████████████████████████                                    | 674/959 [00:02<00:01, 284.33it/s][16:27:46] Unusual charge on atom 17 number of radical electrons set to zero
[16:27:46] Unusual charge on atom 17 number of radical electrons set to zero
100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 959/959 [00:03<00:00, 293.97it/s]


In [9]:
df

,index,mean,25%,50%,75%,5A,2A,median
0,TankBind_RMSD,16.774886,10.766445,14.707578,19.703259,0.938478,0.000000,14.707578
1,TankBind_COM_DIST,15.055478,8.796394,12.864481,17.973485,5.630865,0.625652,12.864481


In [13]:
from tankbind_philip.TankBind.tankbind.data import TankBindDataSet, TankBindDataLoader

In [12]:
dataset = TankBindDataSet("/fs/pool/pool-marsot/pdbbind/pdbbind2020/dataset")

['/fs/pool/pool-marsot/pdbbind/pdbbind2020/dataset/processed/data.pt', '/fs/pool/pool-marsot/pdbbind/pdbbind2020/dataset/processed/protein.pt', '/fs/pool/pool-marsot/pdbbind/pdbbind2020/dataset/processed/compound.pt']


In [24]:
dataloader = TankBindDataLoader(dataset, batch_size=1, shuffle=False, num_workers=8, follow_batch=['x', 'compound_pair'])

In [25]:
from tqdm.notebook import tqdm
for batch in tqdm(dataloader):
    None

  0%|          | 0/164454 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [19]:
batch = next(iter(dataloader))

In [20]:
batch

HeteroDataBatch(
  dis_map=[12948],
  node_xyz=[996, 3],
  coords=[104, 3],
  y=[12948],
  seq=[996],
  affinity=[8],
  compound_pair=[1352, 16],
  compound_pair_batch=[1352],
  compound_pair_ptr=[9],
  pdb=[8],
  group=[8],
  real_affinity_mask=[8],
  real_y_mask=[12948],
  is_equivalent_native_pocket=[8],
  equivalent_native_y_mask=[12948],
  batch_n=8,
  max_dim_divisible_by_8_protein=192,
  max_dim_divisible_by_8_compound=16,
  protein={
    node_s=[996, 6],
    node_v=[996, 3, 3],
    batch=[996],
    ptr=[9],
  },
  compound={
    x=[104, 56],
    x_batch=[104],
    x_ptr=[9],
    batch=[104],
    ptr=[9],
  },
  (protein, p2p, protein)={
    edge_index=[2, 24618],
    edge_s=[24618, 32],
    edge_v=[24618, 1, 3],
    pairwise_representation=[8, 192, 192],
  },
  (compound, c2c, compound)={
    edge_index=[2, 192],
    edge_weight=[192],
    edge_attr=[192, 19],
  },
  (compound, p2p, compound)={
    pairwise_representation=[8, 16, 16, 16],
    pairwise_representation_mask=[8, 16

list of features to normalize:
- protein node_s, node_v
- compound 

In [37]:
batch["protein", "p2p", "protein"]

{'edge_index': tensor([[  1,  47,  43,  ...,  41,  42, 107],
        [  0,   0,   0,  ..., 124, 124, 124]]), 'edge_s': tensor([[ 9.1701e-05,  1.9643e-02,  4.3231e-01,  ...,  3.1623e-03,
          1.0000e-03,  3.1623e-04],
        [ 4.1132e-05,  1.1596e-02,  3.3588e-01,  ...,  1.4808e-01,
          4.6983e-02,  1.4862e-02],
        [ 3.2799e-06,  2.0610e-03,  1.3305e-01,  ...,  1.3556e-01,
          4.2987e-02,  1.3597e-02],
        ...,
        [ 1.8570e-24,  4.2017e-18,  9.7673e-13,  ..., -3.3194e-01,
         -1.0680e-01, -3.3830e-02],
        [ 9.8939e-25,  2.4509e-18,  6.2375e-13,  ..., -3.2896e-01,
         -1.0580e-01, -3.3514e-02],
        [ 1.8871e-26,  8.1695e-20,  3.6336e-14,  ..., -7.8975e-02,
         -2.4997e-02, -7.9056e-03]]), 'edge_v': tensor([[[-0.0488, -0.1734, -0.9836]],

        [[ 0.2769,  0.6440,  0.7132]],

        [[ 0.7999,  0.1222, -0.5876]],

        ...,

        [[ 0.6104,  0.6225, -0.4898]],

        [[ 0.2378,  0.7674, -0.5954]],

        [[ 0.7234, -0.68

In [36]:
batch["compound", "c2c", "compound"]

{'edge_index': tensor([[ 0,  1,  1,  2,  2,  3,  3,  4,  4,  5,  5,  6,  6,  7,  7,  8,  8,  9,
          9, 10, 10, 11, 11, 12, 11, 13, 11, 14,  8, 15, 15, 16, 15, 17, 17, 18,
          5, 19, 19, 20, 19, 21, 21,  2, 17,  6],
        [ 1,  0,  2,  1,  3,  2,  4,  3,  5,  4,  6,  5,  7,  6,  8,  7,  9,  8,
         10,  9, 11, 10, 12, 11, 13, 11, 14, 11, 15,  8, 16, 15, 17, 15, 18, 17,
         19,  5, 20, 19, 21, 19,  2, 21,  6, 17]]), 'edge_weight': tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]), 'edge_attr': tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 1.0000,
         0.0000, 0.0000, 1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         1.5000],
        [1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 1.0000,
         0.0000, 0.0000, 1.0000, 0.0000, 0.0000, 0.0000, 0.0

In [35]:
batch["compound"]

{'x': tensor([[1, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 1,  ..., 0, 0, 1],
        ...,
        [0, 0, 1,  ..., 0, 0, 1],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 1]]), 'x_batch': tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]), 'x_ptr': tensor([ 0, 22]), 'batch': tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]), 'ptr': tensor([ 0, 22])}

In [33]:
batch["protein"]

{'node_s': tensor([[-0.7856, -0.8922, -0.9989, -0.6187,  0.4517, -0.0465],
        [-0.7120, -0.7839, -0.9991, -0.7022,  0.6209,  0.0419],
        [-0.7375, -0.8748, -0.9998, -0.6753,  0.4844,  0.0202],
        [-0.8583, -0.6825, -0.9974, -0.5131,  0.7309, -0.0720],
        [-0.8229, -0.8310, -0.9999, -0.5682,  0.5563,  0.0167],
        [-0.7437, -0.6785, -0.9714, -0.6685,  0.7346,  0.2375],
        [-0.5988, -0.8159, -0.9987, -0.8009,  0.5781,  0.0507],
        [ 0.0321, -0.8760, -0.9880, -0.9995,  0.4823,  0.1547],
        [ 0.3429, -0.7669, -0.9871, -0.9394,  0.6417,  0.1599],
        [-0.9853, -0.9451, -0.9690, -0.1708,  0.3267, -0.2470],
        [ 0.7532, -0.6964, -0.9289,  0.6578, -0.7177, -0.3704],
        [-0.5011, -0.8490, -0.9962, -0.8654,  0.5285,  0.0874],
        [ 0.1306, -0.9823,  0.8917, -0.9914,  0.1876,  0.4527],
        [-0.0188, -0.9809, -0.9746, -0.9998,  0.1945,  0.2241],
        [-0.0547, -0.9512, -0.9972, -0.9985,  0.3085,  0.0751],
        [-0.5687, -0.4928, -0

In [32]:
batch["protein"].node_v

tensor([[[-0.0488, -0.1734, -0.9836],
         [-0.4420, -0.3378,  0.8310],
         [ 0.9138,  0.3623,  0.1835]],

        [[ 0.6216,  0.2574, -0.7398],
         [ 0.0488,  0.1734,  0.9836],
         [-0.9455, -0.0213, -0.3251]],

        [[-0.0239, -0.0725, -0.9971],
         [-0.6216, -0.2574,  0.7398],
         [ 0.9606, -0.0264,  0.2765]],

        ...,

        [[-0.4515, -0.0144,  0.8922],
         [-0.2233,  0.0297, -0.9743],
         [ 0.8842,  0.4432,  0.1474]],

        [[ 0.3521,  0.2108,  0.9119],
         [ 0.4515,  0.0144, -0.8922],
         [-0.7370, -0.6695,  0.0923]],

        [[-0.2495, -0.0482,  0.9672],
         [-0.3521, -0.2108, -0.9119],
         [ 0.6211,  0.7809, -0.0662]]])

In [23]:
batch.node_xyz

tensor([[ 1.4570, 36.3910, 19.4710],
        [ 5.0260, 36.6890, 18.1630],
        [ 5.8470, 36.3800, 14.4620],
        ...,
        [ 7.5770, 45.2690, 29.6660],
        [ 9.1140, 48.7290, 30.1670],
        [ 7.2050, 50.7510, 27.5410]])

In [26]:
batch.dis_map

tensor([10., 10., 10.,  ..., 10., 10., 10.])

In [27]:
batch.coords

tensor([[ 74.8174, -64.9541,  41.1500],
        [ 73.8850, -65.3640,  41.2040],
        [ 73.7210, -66.5660,  41.7280],
        [ 72.4440, -67.0920,  41.8140],
        [ 72.2460, -68.3500,  42.3740],
        [ 73.3350, -69.0940,  42.8540],
        [ 73.1400, -70.4510,  43.4510],
        [ 72.0040, -71.1110,  42.8400],
        [ 71.1620, -71.6210,  43.8930],
        [ 69.7030, -71.6540,  43.4090],
        [ 69.2900, -70.4020,  42.8180],
        [ 68.0490, -70.4000,  41.7900],
        [ 67.5360, -69.0370,  41.4610],
        [ 68.5660, -71.1910,  40.4760],
        [ 66.8870, -71.3290,  42.4460],
        [ 71.3660, -70.6440,  45.0360],
        [ 71.0310, -71.2810,  46.2560],
        [ 72.8610, -70.4070,  44.9580],
        [ 73.2050, -69.1220,  45.4570],
        [ 74.6270, -68.5510,  42.7750],
        [ 75.5820, -69.1970,  43.2040],
        [ 74.8210, -67.2830,  42.2110]])

In [28]:
batch.affinity

tensor([1.7200])